## Deduplication quickstart

In this demo we de-duplicate a small dataset, using simple settings. The aim is to demonstarate core Splink functionality succinctly, rather that comprehensively document all configuration options.



## Step 1: Choose your backend

In `splink` version 3, you have the option to choose the SQL backend that will perform match

Currently, `splink` offers three different SQL backends:
- `duckdb` with: `from splink.duckdb.duckdb_linker import DuckDBLinker`

- `sqlite` with: `from splink.sqlite.sqlite_linker import SQLiteLinker`

- `spark` with: `from splink.spark.spark_linker import SparkLinker`

For smaller datasets (up to a few million records), we `duckdb` is likely to give you the best performance.

The subsequent code is the same irrespective of the backend used.

In [1]:
from splink.duckdb.duckdb_linker import DuckDBLinker

## Step 1: Read in data
Read in a 1000-record dataset that contains duplicates.

Note that the `group` column represents the 'ground truth' - i.e. this is a labelled dataset, so we know which rows refer to the same person.  In reality, we wouldn't have this column - this is the information that Splink is trying to estimate.

In [2]:
import pandas as pd 
pd.options.display.max_rows = 1000
df = pd.read_csv("./data/fake_1000.csv")
df.head(5)

,unique_id,first_name,surname,dob,city,email,group
0,0,Robert,Alan,1971-06-24,NaN,robert255@smith.net,0
1,1,Robert,Allen,1971-05-24,NaN,roberta25@smith.net,0
2,2,Rob,Allen,1971-06-24,London,roberta25@smith.net,0
3,3,Robert,Alen,1971-06-24,Lonon,NaN,0
4,4,Grace,NaN,1997-04-26,Hull,grace.kelly52@jones.com,1


## Step 2: Profile columns

Splink can perform exploratory analysis of columns (e.g. `first_name`, or arbitrary sql expressions like `concat(first_name, surname)`).  

This is useful for understanding your data, whether it suffers from skew, and whether additional data cleaning may be necessary.

In [3]:
# Initialise the linker, passing in the input dataset(s)
linker = DuckDBLinker(input_tables = {"fake_1000": df})

linker.profile_columns(["first_name", "city", "substr(dob, 1,4)"], top_n=10, bottom_n=5)

## Step 3: Configure how Splink compares records using a `settings` dictionary

`splink` needs to know how to compare records from the input dataset:  Which columns should be compared, and how should Splink assess their similarity?

This is configured using a `settings` dictionary.  For the purposes of this simple example, we will make these comparisons simple:  

- For the `first_name` column, we will model the comparison as either:
  - an 'exact match' (e.g. `John` vs `John`)
  - similar but not the exactly the same (e.g. `John` vs `Jon`).  Specifically this will be defined as a levenshtein distance of either 1 or 2.
  - all other comparisons 

- For all other copmarisons, Splink will categorise comparisons as either an 'exact match' (e.g. `Smith` vs `Smith`), or 'anything else' (e.g. `Smith` vs `Jones`, or even `Smith` vs `Smyth`).

- For `city`, we enable term frequency comparisons because we observed significant skew in the distribution of values

In [4]:

from splink.comparison_library import exact_match, levenshtein
settings = {
    "proportion_of_matches": 0.01,
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name",
        "l.surname = r.surname",
    ],
    "comparisons": [
        levenshtein("first_name", 2),
        exact_match("surname"),
        exact_match("dob"),
        exact_match("city", term_frequency_adjustments=True),
        exact_match("email"),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "additional_columns_to_retain": ["group"],
    "max_iterations": 10,
    "em_convergence": 0.01
}

In words, this setting dictionary says:

* We are performing a deduplication task (the other options are `link_only`, or `link_and_dedupe`, which may be used if there are multiple input datasets)
* The blocking rule states that we will only check for duplicates amongst records where the `first_name`s or `surname`s are identical.
* When comparing records, we will use information from the first_name, surname, dob, city and email columns to compute a match score.
* We have enabled term frequency adjustments for the 'city' column, because some values (e.g. `London`) appear much more frequently than others
* We will retain the group column in the results even though this is not used as part of comparisons. This is a labelled dataset and group contains the true match status, so it is interesting to retain this information so it can be compared to the Splink estimates.
* Will will consider the algorithm to have converged when no parameter changes by more than 0.01 between iterations.
* To ensure the notebook runs quickly, we will stop iterations at 10.
* We have set `retain_intermediate_calculation_columns` and `additional_columns_to_retain` to True for the purposes of the demo, because this will mean the output datasets contain additional information that, whilst not strictly needed by Splink, helps the user understand the calculations. If these were not included in the settings dictionary, they would be set to `False` (their default value).

## Step 4: Estimate the parameters of your model

Estimate the parameters of a Fellegi Sunter model, and use the model to generate predictions.  

We starby by using the `train_u_using_random_sampling` to compute the `u` values of the model.




In [5]:
linker.initialise_settings(settings)
linker.train_u_using_random_sampling(target_rows=1e6)

We then use the expectation maximisation algorithm to train the `m` values.

Note that in this first EM training session we block on `first_name` and `surname`, meaning that all comparisons will have `first_name` and `surname` exactly equal.   This means that, in this training session, we cannot estimate parameter estimates for the `first_name` or `surname` columns, as seen in their absence from the match weights chart.

In [6]:
blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_names = linker.train_m_using_expectation_maximisation(blocking_rule)


training_session_names.match_weights_interactive_history_chart()

Iteration 0: Largest change in params was 0.475 in the m_probability of dob, level `All other comparisons`
Iteration 1: Largest change in params was -0.016 in the m_probability of city, level `exact_match`
Iteration 2: Largest change in params was 0.00595 in proportion_of_matches
EM converged after 2 iterations
Proportion of matches not fully trained, current estimates are [0.0004748287721487068]


In a second training session, we block on `dob`.  This allows us to estimate parameters for the `first_name ` and `surname` comparisons.

Between the two training sessions, we now have parameter estimates for all comparisons.

In [7]:
blocking_rule = "l.dob = r.dob"
training_session_dob = linker.train_m_using_expectation_maximisation(blocking_rule)
training_session_names.match_weights_interactive_history_chart()

Iteration 0: Largest change in params was 0.51 in proportion_of_matches
Iteration 1: Largest change in params was 0.18 in proportion_of_matches
Iteration 2: Largest change in params was 0.0618 in proportion_of_matches
Iteration 3: Largest change in params was 0.0286 in proportion_of_matches
Iteration 4: Largest change in params was 0.0159 in proportion_of_matches
Iteration 5: Largest change in params was 0.00983 in proportion_of_matches
EM converged after 5 iterations
Proportion of matches can now be estimated, estimates are [0.001301378322364649, 0.03974577977000298]


The final match weights can be viewed in the match weights chart:

In [8]:
linker.settings_obj.match_weights_chart()

## Step 7: Predicting match weights using the trained model 

In [9]:
df_predictions = linker.predict()

## Step 8: Visualising results

We can view the output table as follows:

In [10]:
df_predictions.as_pandas_dataframe().head(5)

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,surname_l,surname_r,...,tf_city_r,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,bf_email,group_l,group_r,match_key
0,8.156673,0.996508,4,5,Grace,Grace,2,87.219312,NaN,Kelly,...,NaN,1.000000,1.0,grace.kelly52@jones.com,grace.kelly52@jones.com,1,268.445807,1,1,0
1,-2.058547,0.193586,9,922,Evie,Evie,2,87.219312,Dean,Jones,...,NaN,1.000000,1.0,evihd56@earris-bailey.net,eviejones@brewer-sparks.org,0,0.411968,3,230,0
2,-2.058547,0.193586,14,998,Oliver,Oliver,2,87.219312,Griffiths,Bird,...,NaN,1.000000,1.0,o.griffiths90@reyes-coleman.com,oliver.b@smith.net,0,0.411968,5,250,0
3,-0.779152,0.368172,18,475,Caleb,Caleb,2,87.219312,Rwoe,Scott,...,NaN,1.000000,1.0,NaN,c.scott@brooks.com,-1,1.000000,8,119,0
4,-3.262077,0.094396,21,917,Darcy,Darcy,2,87.219312,Bernass,Rhodes,...,0.0492,0.434211,1.0,darcy.b@silva.com,drhodes16@johnson-robinson.com,0,0.411968,9,229,0


You can also view rows in this dataset as a waterfall chart as follows:

In [11]:
from splink.charts import waterfall_chart
records_to_plot = df_predictions.as_pandas_dataframe().head(5).to_dict(orient="records")
waterfall_chart(records_to_plot, linker.settings_obj, filter_nulls=False)

A histogram showing the distribution of match weights can be viewed as follows

In [12]:
linker.match_weight_histogram(df_predictions)

If you have a sample of labels, you can output a ROC chart.  (A precision-recall chart is also available with `linker.precision_recall_from_labels`) 

Your labels need to be formatted as follows:

In [13]:
df_labels = pd.read_csv("./data/fake_1000_labels.csv")
df_labels.head(5)



,unique_id_l,source_dataset_l,unique_id_r,source_dataset_r,clerical_match_score
0,0,fake_1000,1,fake_1000,1.0
1,0,fake_1000,2,fake_1000,1.0
2,0,fake_1000,3,fake_1000,1.0
3,0,fake_1000,4,fake_1000,0.0
4,0,fake_1000,5,fake_1000,0.0


Then to produce the chart:

In [14]:
linker.con.register("labels", df_labels)

linker.roc_from_labels("labels")